In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-forest-heading-classifier/other/default/1/model.pkl
/kaggle/input/test23231/testpdf.pdf
/kaggle/input/dataset12345/1adataset/Adobe2/UIAWYVO3ZI6VIPTWV6TB7CFFNGCQ23PA.json
/kaggle/input/dataset12345/1adataset/Adobe2/QNKRV7UHP5TEQSOYA73EJYJ7MSG66S3D.json
/kaggle/input/dataset12345/1adataset/Adobe2/TU5MAMSG2RMTDZQDKBULDLWSVH6GSYHX.pdf.json
/kaggle/input/dataset12345/1adataset/Adobe2/KNB57BFSJINB7DC3WJQ452LPY5C7L7UL.json
/kaggle/input/dataset12345/1adataset/Adobe2/LDLCDEPVQ225ADIF33NSEBYX3Z3AE4TX.json
/kaggle/input/dataset12345/1adataset/Adobe2/SH2IQH724IFKG2K3GQ7RCDJ6Q7IJQLGE.json
/kaggle/input/dataset12345/1adataset/Adobe2/RBQS77ZEQXZZN5PHJPGI4PI4XB3ANVBB.json
/kaggle/input/dataset12345/1adataset/Adobe2/OB54TYALNJNUFASVPJ6FQ4ZJGVQ2FJMM.json
/kaggle/input/dataset12345/1adataset/Adobe2/PLOHNC5FBG7GG7BN7IZ5TLOLYMGRDOY6.json
/kaggle/input/dataset12345/1adataset/Adobe2/RLKGPI2MJLO7AEAI772IBBKWBRRWISBF.json
/kaggle/input/dataset12345/1adataset/Adobe2/QZ74K7KFE4OPYKEUIOXGTTM

# Heuristics

In [14]:
import pdfplumber
import json
import re
from collections import Counter
from pathlib import Path

def classify_line(text, font_size, font_name, top_y, page_num, page_font_sizes):
    text = text.strip()

    # Rule: Title
    if page_num == 1 and font_size == max(page_font_sizes) and len(text) < 80:
        return 'Title'

    # Rule: H1
    if font_size >= sorted(set(page_font_sizes), reverse=True)[0] and len(text) < 80 and not text.endswith('.'):
        if text.isupper() or text.istitle():
            return 'H1'

    # Rule: H2
    if re.match(r'^\d+\.\d+', text) and len(text) < 100:
        return 'H2'

    # Rule: Paragraph
    if len(text.split()) > 5 and text[-1] in '.!?':
        return 'Body'

    # Fallback
    return 'Body'


def extract_pdf_to_json(pdf_path):
    output = {"pages": []}

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            lines = page.extract_words(extra_attrs=['fontname', 'size', 'top'])
            if not lines:
                continue

            page_data = []
            page_font_sizes = [round(word['size']) for word in lines]
            content_by_y = {}

            for word in lines:
                y = round(word['top'], 1)
                if y not in content_by_y:
                    content_by_y[y] = []
                content_by_y[y].append(word)

            current_section = None
            current_subsection = None

            for y in sorted(content_by_y.keys(), reverse=True):
                words = sorted(content_by_y[y], key=lambda w: w['x0'])
                line_text = " ".join([w['text'] for w in words])
                font_sizes = [round(w['size']) for w in words]
                font_names = [w['fontname'] for w in words]
                avg_font_size = round(sum(font_sizes) / len(font_sizes), 1)
                most_common_font = Counter(font_names).most_common(1)[0][0]

                content_type = classify_line(line_text, avg_font_size, most_common_font, y, page_num, page_font_sizes)

                if content_type == 'Title':
                    continue  # skip adding title to body

                elif content_type == 'H1':
                    current_section = line_text
                    current_subsection = None

                elif content_type == 'H2':
                    current_subsection = line_text

                else:
                    page_data.append({
                        "type": "paragraph",
                        "section": current_section,
                        "sub_section": current_subsection,
                        "text": line_text
                    })

            # Add tables
            for table in page.extract_tables():
                if table:
                    page_data.append({
                        "type": "table",
                        "section": current_section,
                        "sub_section": current_subsection,
                        "description": None,
                        "table_data": table
                    })

            # Add images as charts
            for image in page.images:
                page_data.append({
                    "type": "chart",
                    "section": current_section,
                    "sub_section": current_subsection,
                    "description": f"Chart/Image at y={image['top']}",
                    "table_data": []
                })

            if page_data:
                output['pages'].append({
                    "page_number": page_num,
                    "content": page_data
                })

    return output

In [15]:
pdf_path = "/kaggle/input/test23231/testpdf.pdf"
output_path = "output_heusristic.json"
parsed = extract_pdf_to_json(pdf_path)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(parsed, f, indent=2, ensure_ascii=False)

print(f"✅ JSON saved to {output_path}")

✅ JSON saved to output_heusristic.json


In [10]:
import json
import os
import re
import pickle
from pathlib import Path
from typing import List, Dict, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

import pdfplumber
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib

In [16]:
!pip install pdfplumber

In [12]:


class PDFContentExtractor:
    def __init__(self):
        self.classifier = None
        self.label_encoder = LabelEncoder()
        self.feature_columns = []
        
    def extract_text_with_layout(self, pdf_path: str) -> List[Dict]:
        """Extract text tokens with layout information from PDF"""
        lines = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):  # Start from page 1
                # Extract text with layout information
                words = page.extract_words()
                
                if not words:
                    continue
                
                # Group words into lines based on y-coordinate proximity
                words_by_line = {}
                for word in words:
                    y_coord = round(word['top'], 1)  # Round to group nearby words
                    if y_coord not in words_by_line:
                        words_by_line[y_coord] = []
                    words_by_line[y_coord].append(word)
                
                # Process each line
                for y_coord in sorted(words_by_line.keys()):
                    line_words = sorted(words_by_line[y_coord], key=lambda x: x['x0'])
                    
                    if not line_words:
                        continue
                    
                    # Merge words into line text
                    line_text = ' '.join([word['text'] for word in line_words])
                    line_text = line_text.strip()
                    
                    if len(line_text) < 2:  # Skip very short lines
                        continue
                    
                    # Calculate line properties
                    avg_size = np.mean([word.get('size', 12) for word in line_words])
                    font_names = [word.get('fontname', '') for word in line_words]
                    most_common_font = max(set(font_names), key=font_names.count) if font_names else ''
                    
                    line_info = {
                        'text': line_text,
                        'page': page_num,
                        'y_coord': y_coord,
                        'x_coord': min([word['x0'] for word in line_words]),
                        'font_size': avg_size,
                        'font_name': most_common_font,
                        'bbox': {
                            'x0': min([word['x0'] for word in line_words]),
                            'y0': min([word['top'] for word in line_words]),
                            'x1': max([word['x1'] for word in line_words]),
                            'y1': max([word['bottom'] for word in line_words])
                        }
                    }
                    
                    lines.append(line_info)
        
        return lines

    def extract_tables(self, pdf_path: str) -> List[Dict]:
        """Extract tables from PDF"""
        tables = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                # Extract tables from the page
                page_tables = page.extract_tables()
                
                for table_idx, table in enumerate(page_tables):
                    if table and len(table) > 1:  # Ensure table has content
                        # Clean the table data
                        cleaned_table = []
                        for row in table:
                            cleaned_row = [cell.strip() if cell else "" for cell in row]
                            if any(cleaned_row):  # Skip empty rows
                                cleaned_table.append(cleaned_row)
                        
                        if cleaned_table:
                            tables.append({
                                'page': page_num,
                                'table_data': cleaned_table,
                                'y_coord': 0,  # Could be improved to get actual position
                                'type': 'table'
                            })
        
        return tables

    def extract_charts(self, pdf_path: str) -> List[Dict]:
        """Extract chart information from PDF (basic detection)"""
        charts = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                # Check for images (potential charts)
                if hasattr(page, 'images') and page.images:
                    for img_idx, img in enumerate(page.images):
                        charts.append({
                            'page': page_num,
                            'type': 'chart',
                            'description': f"Chart/Image found on page {page_num}",
                            'y_coord': img.get('top', 0),
                            'table_data': []  # Charts might have associated data
                        })
                
                # Also check for chart-like keywords in text
                words = page.extract_words()
                chart_keywords = ['figure', 'chart', 'graph', 'diagram', 'plot']
                
                for word in words:
                    if any(keyword in word['text'].lower() for keyword in chart_keywords):
                        # Found potential chart reference
                        charts.append({
                            'page': page_num,
                            'type': 'chart',
                            'description': f"Chart reference: {word['text']}",
                            'y_coord': word['top'],
                            'table_data': []
                        })
                        break  # Only add one per page to avoid duplicates
        
        return charts
    
    def compute_features(self, lines: List[Dict]) -> pd.DataFrame:
        """Compute features for each line"""
        features = []
        
        # Group lines by page for relative calculations
        pages = {}
        for line in lines:
            page_num = line['page']
            if page_num not in pages:
                pages[page_num] = []
            pages[page_num].append(line)
        
        for line in lines:
            page_num = line['page']
            page_lines = pages[page_num]
            
            # Basic text features
            text = line['text']
            word_count = len(text.split())
            char_count = len(text)
            uppercase_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
            
            # Font size features (relative to page)
            page_font_sizes = [l['font_size'] for l in page_lines]
            font_size_rank = sorted(set(page_font_sizes), reverse=True).index(line['font_size']) + 1
            font_size_percentile = np.percentile(page_font_sizes, 75) if len(page_font_sizes) > 1 else line['font_size']
            is_large_font = line['font_size'] >= font_size_percentile
            
            # Position features
            page_x_coords = [l['x_coord'] for l in page_lines]
            median_x = np.median(page_x_coords)
            indentation = abs(line['x_coord'] - median_x)
            
            # Whitespace features (simplified)
            y_coords = sorted([l['y_coord'] for l in page_lines])
            line_y_idx = y_coords.index(line['y_coord'])
            
            space_above = 0
            space_below = 0
            if line_y_idx > 0:
                space_above = abs(y_coords[line_y_idx] - y_coords[line_y_idx - 1])
            if line_y_idx < len(y_coords) - 1:
                space_below = abs(y_coords[line_y_idx + 1] - y_coords[line_y_idx])
            
            # Text pattern features
            has_numbering = bool(re.match(r'^\d+[\.\):]', text.strip()))
            has_sub_numbering = bool(re.match(r'^\d+\.\d+', text.strip()))
            is_all_caps = text.isupper() and len(text) > 3
            ends_with_colon = text.strip().endswith(':')
            
            # Font style features (basic)
            is_bold = 'bold' in line['font_name'].lower() or 'Bold' in line['font_name']
            is_italic = 'italic' in line['font_name'].lower() or 'Italic' in line['font_name']
            
            # Position in document
            is_first_page = page_num == 1
            relative_position = (page_num - 1) / max(len(pages), 1)
            
            feature_dict = {
                'word_count': word_count,
                'char_count': char_count,
                'uppercase_ratio': uppercase_ratio,
                'font_size': line['font_size'],
                'font_size_rank': font_size_rank,
                'is_large_font': int(is_large_font),
                'indentation': indentation,
                'space_above': space_above,
                'space_below': space_below,
                'has_numbering': int(has_numbering),
                'has_sub_numbering': int(has_sub_numbering),
                'is_all_caps': int(is_all_caps),
                'ends_with_colon': int(ends_with_colon),
                'is_bold': int(is_bold),
                'is_italic': int(is_italic),
                'is_first_page': int(is_first_page),
                'relative_position': relative_position,
                'page': page_num,
                'text': text
            }
            
            features.append(feature_dict)
        
        return pd.DataFrame(features)
    
    def align_with_ground_truth(self, lines_df: pd.DataFrame, ground_truth: Dict) -> pd.DataFrame:
        """Align extracted lines with ground truth labels"""
        lines_df['label'] = 'Body'  # Default label
        
        # Debug: Print the structure of ground_truth
        print(f"Ground truth keys: {list(ground_truth.keys())}")
        if 'outline' in ground_truth and len(ground_truth['outline']) > 0:
            print(f"First outline entry keys: {list(ground_truth['outline'][0].keys())}")
        
        # Label title
        if 'title' in ground_truth:
            title_text = ground_truth['title']
            if title_text:  # Only process if title is not empty
                title_matches = lines_df['text'].str.contains(re.escape(title_text), case=False, na=False)
                if title_matches.any():
                    lines_df.loc[title_matches, 'label'] = 'Title'
                else:
                    # Try fuzzy matching for title
                    for idx, row in lines_df.iterrows():
                        if row['page'] == 1 and len(set(title_text.lower().split()) & set(row['text'].lower().split())) >= 2:
                            lines_df.loc[idx, 'label'] = 'Title'
                            break
        
        # Label headings - handle different JSON structures
        if 'outline' in ground_truth:
            for heading in ground_truth['outline']:
                # Handle different possible key names
                heading_text = heading.get('text', heading.get('heading', ''))
                heading_page = heading.get('page', heading.get('page_number', 1))
                
                # Try different ways to get the level
                if 'level' in heading:
                    heading_level = heading['level']
                elif 'type' in heading:
                    heading_level = heading['type']
                elif 'heading_level' in heading:
                    heading_level = f"H{heading['heading_level']}"
                else:
                    # Try to infer from other fields or default to H1
                    print(f"Warning: Could not determine level for heading: {heading}")
                    heading_level = 'H1'  # Default fallback
                
                # Skip if no text
                if not heading_text:
                    continue
                
                # Find matching lines on the same page
                page_lines = lines_df[lines_df['page'] == heading_page]
                
                # Exact match first
                exact_matches = page_lines['text'].str.contains(re.escape(heading_text), case=False, na=False)
                if exact_matches.any():
                    matching_idx = page_lines[exact_matches].index[0]
                    lines_df.loc[matching_idx, 'label'] = heading_level
                else:
                    # Fuzzy match
                    best_match_idx = None
                    best_similarity = 0
                    
                    for idx, row in page_lines.iterrows():
                        # Simple word overlap similarity
                        heading_words = set(heading_text.lower().split())
                        line_words = set(row['text'].lower().split())
                        similarity = len(heading_words & line_words) / max(len(heading_words), 1)
                        
                        if similarity > best_similarity and similarity > 0.5:
                            best_similarity = similarity
                            best_match_idx = idx
                    
                    if best_match_idx is not None:
                        lines_df.loc[best_match_idx, 'label'] = heading_level
        
        return lines_df
    
    def train_model(self, train_data_dir: str, ground_truth_dir: str):
        """Train the classifier using training data"""
        all_features = []
        all_labels = []
        
        # Process all training files
        pdf_files = list(Path(train_data_dir).glob('*.pdf'))
        successful_files = 0
        failed_files = 0
        
        print(f"Found {len(pdf_files)} PDF files to process")
        
        for pdf_file in pdf_files:
            json_file = Path(ground_truth_dir) / f"{pdf_file.stem}.json"
            
            if not json_file.exists():
                print(f"❌ Ground truth not found for {pdf_file.name}")
                failed_files += 1
                continue
            
            print(f"📄 Processing {pdf_file.name}...")
            
            try:
                # Load ground truth
                with open(json_file, 'r', encoding='utf-8') as f:
                    ground_truth = json.load(f)
                
                # Extract lines from PDF
                lines = self.extract_text_with_layout(str(pdf_file))
                if not lines:
                    print(f"⚠️   No text extracted from {pdf_file.name}")
                    failed_files += 1
                    continue
                    
                # Compute features
                lines_df = self.compute_features(lines)
                
                # Align with ground truth
                lines_df = self.align_with_ground_truth(lines_df, ground_truth)
                
                # Check if we got any labeled data
                non_body_labels = lines_df[lines_df['label'] != 'Body']
                if len(non_body_labels) == 0:
                    print(f"⚠️   No headings/title found in {pdf_file.name}")
                
                # Add to training data
                feature_cols = [col for col in lines_df.columns if col not in ['text', 'label']]
                all_features.append(lines_df[feature_cols])
                all_labels.extend(lines_df['label'].tolist())
                
                successful_files += 1
                print(f"✅ Successfully processed {pdf_file.name}")
                
            except Exception as e:
                print(f"❌ Error processing {pdf_file.name}: {e}")
                failed_files += 1
                import traceback
                print(f"   Traceback: {traceback.format_exc()}")
                continue
        
        print(f"\n📊 Training Summary:")
        print(f"   Successful files: {successful_files}")
        print(f"   Failed files: {failed_files}")
        print(f"   Total training samples: {len(all_labels)}")
        
        if not all_features:
            raise ValueError("❌ No training data found! All files failed to process.")
        
        # Combine all features
        try:
            X = pd.concat(all_features, ignore_index=True)
            y = np.array(all_labels)
            
            print(f"   Feature matrix shape: {X.shape}")
            
            # Store feature columns
            self.feature_columns = X.columns.tolist()
            
            # Encode labels
            y_encoded = self.label_encoder.fit_transform(y)
            
            # Check if we have multiple classes
            unique_classes = len(np.unique(y_encoded))
            if unique_classes < 2:
                print(f"⚠️   Warning: Only {unique_classes} unique class found. Model may not work well.")
            
            # Train model
            self.classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                class_weight='balanced'
            )
            
            # Split for validation if we have enough data
            if len(X) > 10 and unique_classes > 1:
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
                )
                
                # Train
                self.classifier.fit(X_train, y_train)
                
                # Validate
                y_pred = self.classifier.predict(X_val)
                accuracy = accuracy_score(y_val, y_pred)
                print(f"✅ Validation Accuracy: {accuracy:.3f}")
            else:
                print("⚠️   Too few samples for validation split, training on all data")
                self.classifier.fit(X, y_encoded)
            
            # Show label distribution
            unique_labels, counts = np.unique(y, return_counts=True)
            print("\n📋 Label distribution:")
            for label, count in zip(unique_labels, counts):
                print(f"   {label}: {count}")
                
            print("✅ Training completed successfully!")
            
        except Exception as e:
            print(f"❌ Error during model training: {e}")
            import traceback
            print(f"   Traceback: {traceback.format_exc()}")
            raise
    
    def detect_content_type(self, text: str, font_size: float, is_heading: bool = False) -> str:
        """Detect if text is a paragraph based on characteristics"""
        if is_heading:
            return "heading"
        
        # Simple paragraph detection - could be enhanced
        if len(text.split()) > 3:  # More than 3 words likely a paragraph
            return "paragraph"
        
        return "paragraph"  # Default to paragraph for text content

    def parse_pdf_to_json(self, pdf_path: str) -> Dict:
        """Parse PDF and return structured JSON for assignment"""
        
        # Extract all content types
        print(f"📄 Processing {pdf_path}...")
        
        # Extract text lines
        lines = self.extract_text_with_layout(pdf_path)
        tables = self.extract_tables(pdf_path)
        charts = self.extract_charts(pdf_path)
        
        if not lines and not tables and not charts:
            return {"pages": []}
        
        # If we have a trained model, use it to detect headings
        if self.classifier is not None and lines:
            lines_df = self.compute_features(lines)
            X = lines_df[self.feature_columns]
            y_pred_encoded = self.classifier.predict(X)
            y_pred = self.label_encoder.inverse_transform(y_pred_encoded)
            
            # Add predictions to lines
            for i, line in enumerate(lines):
                line['predicted_type'] = y_pred[i]
        else:
            # Fallback: simple rule-based heading detection
            for line in lines:
                text = line['text']
                # Simple heuristics for headings
                if (line['font_size'] > 14 or 
                    text.isupper() or 
                    re.match(r'^\d+[\.\)]\s+[A-Z]', text) or
                    len(text) < 60 and not text.endswith('.')):
                    line['predicted_type'] = 'H1'
                else:
                    line['predicted_type'] = 'Body'
        
        # Organize by pages
        pages_data = {}
        
        # Add text lines to pages
        for line in lines:
            page_num = line['page']
            if page_num not in pages_data:
                pages_data[page_num] = []
            
            line['y_coord'] = line['y_coord']  # For sorting
            pages_data[page_num].append(line)
        
        # Add tables to pages
        for table in tables:
            page_num = table['page']
            if page_num not in pages_data:
                pages_data[page_num] = []
            pages_data[page_num].append(table)
        
        # Add charts to pages
        for chart in charts:
            page_num = chart['page']
            if page_num not in pages_data:
                pages_data[page_num] = []
            pages_data[page_num].append(chart)
        
        # Build final JSON structure
        result = {"pages": []}
        
        for page_num in sorted(pages_data.keys()):
            page_content = []
            
            # Sort content by y_coordinate (top to bottom)
            page_items = sorted(pages_data[page_num], 
                              key=lambda x: x.get('y_coord', 0), 
                              reverse=True)  # Higher y_coord = higher on page
            
            # Track current section/subsection
            current_section = None
            current_subsection = None
            
            for item in page_items:
                if item.get('type') == 'table':
                    # Add table
                    page_content.append({
                        "type": "table",
                        "section": current_section,
                        "sub_section": current_subsection,
                        "description": None,
                        "table_data": item['table_data']
                    })
                    
                elif item.get('type') == 'chart':
                    # Add chart
                    page_content.append({
                        "type": "chart", 
                        "section": current_section,
                        "sub_section": current_subsection,
                        "description": item['description'],
                        "table_data": item.get('table_data', [])
                    })
                    
                else:
                    # Handle text content
                    predicted_type = item.get('predicted_type', 'Body')
                    text = item['text']
                    
                    if predicted_type == 'Title':
                        # Skip title in content (could be handled separately if needed)
                        continue
                    elif predicted_type == 'H1':
                        current_section = text
                        current_subsection = None
                        # Don't add heading as separate content item
                        continue
                    elif predicted_type in ['H2', 'H3']:
                        current_subsection = text
                        # Don't add heading as separate content item
                        continue
                    else:
                        # Add as paragraph
                        page_content.append({
                            "type": "paragraph",
                            "section": current_section,
                            "sub_section": current_subsection,
                            "text": text
                        })
            
            if page_content:  # Only add pages with content
                result["pages"].append({
                    "page_number": page_num,
                    "content": page_content
                })
        
        return result
    
    def save_model(self, model_path: str):
        """Save the trained model"""
        model_data = {
            'classifier': self.classifier,
            'label_encoder': self.label_encoder,
            'feature_columns': self.feature_columns
        }
        joblib.dump(model_data, model_path)
    
    def load_model(self, model_path: str):
        """Load a trained model"""
        if os.path.exists(model_path):
            model_data = joblib.load(model_path)
            self.classifier = model_data['classifier']
            self.label_encoder = model_data['label_encoder']
            self.feature_columns = model_data['feature_columns']
            print(f"✅ Model loaded from {model_path}")
        else:
            print(f"⚠️  Model file {model_path} not found. Using rule-based detection.")

def main():
    """Main function for assignment task"""
    extractor = PDFContentExtractor()
    
    # Try to load pre-trained model if available (optional for assignment)
    model_path = 'model.pkl'
    if os.path.exists(model_path):
        extractor.load_model(model_path)
    
    # For training (only run if training data available)
    if os.path.exists('/kaggle/input/dataset12345/archive (3)/Pdf'):
        print("Training mode available...")
        response = input("Do you want to train the model? (y/n): ").lower().strip()
        if response == 'y':
            extractor.train_model('/kaggle/input/dataset12345/archive (3)/Pdf', '/kaggle/input/dataset12345/1adataset/Adobe2')
            extractor.save_model('model.pkl')
    
    # Assignment task: Process PDF file(s)
    input_path = input("Enter PDF file path (or directory): ").strip()
    
    if not input_path:
        # Default paths for testing
        possible_paths = [
            '',
            './input',
            '.'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                input_path = path
                break
        
        if not input_path:
            print("❌ No input path specified and no default paths found")
            return
    
    input_path = Path(input_path)
    output_dir = Path('./output')
    output_dir.mkdir(exist_ok=True)
    
    # Process single file or directory
    if input_path.is_file() and input_path.suffix.lower() == '.pdf':
        pdf_files = [input_path]
    elif input_path.is_dir():
        pdf_files = list(input_path.glob('*.pdf'))
    else:
        print(f"❌ Invalid path: {input_path}")
        return
    
    if not pdf_files:
        print(f"❌ No PDF files found in {input_path}")
        return
    
    print(f"📁 Found {len(pdf_files)} PDF file(s) to process")
    
    # Process each PDF file
    for pdf_file in pdf_files:
        print(f"\n🔄 Processing {pdf_file.name}...")
        try:
            # Parse PDF to assignment JSON format
            result = extractor.parse_pdf_to_json(str(pdf_file))
            
            # Save result
            output_file = output_dir / f"{pdf_file.stem}.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Saved result to {output_file}")
            
            # Print summary
            total_pages = len(result.get('pages', []))
            total_content = sum(len(page.get('content', [])) for page in result.get('pages', []))
            print(f"   📊 Summary: {total_pages} pages, {total_content} content items")
            
        except Exception as e:
            print(f"❌ Error processing {pdf_file.name}: {e}")
            import traceback
            print(f"   Traceback: {traceback.format_exc()}")
            
            # Create empty output file
            output_file = output_dir / f"{pdf_file.stem}.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump({"pages": []}, f, indent=2)

if __name__ == "__main__":
    main()

✅ Model loaded from model.pkl
Training mode available...


KeyboardInterrupt: Interrupted by user

In [17]:
# ===============================
# 📌 PDF Parsing & JSON Extraction (Kaggle Notebook)
# ===============================

import json
import re
import warnings
from pathlib import Path
from typing import List, Dict
import pdfplumber
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib

warnings.filterwarnings('ignore')


# ===============================
# 📌 PDFOutlineExtractor Class
# ===============================

class PDFOutlineExtractor:
    def __init__(self):
        self.classifier = None
        self.label_encoder = LabelEncoder()
        self.feature_columns = []
        
    def extract_text_with_layout(self, pdf_path: str) -> List[Dict]:
        """Extract text tokens with layout information from PDF"""
        lines = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 0):
                words = page.extract_words()
                if not words:
                    continue
                
                # Group words into lines by y-coordinate
                words_by_line = {}
                for word in words:
                    y_coord = round(word['top'], 1)
                    if y_coord not in words_by_line:
                        words_by_line[y_coord] = []
                    words_by_line[y_coord].append(word)
                
                for y_coord in sorted(words_by_line.keys()):
                    line_words = sorted(words_by_line[y_coord], key=lambda x: x['x0'])
                    if not line_words:
                        continue
                    
                    line_text = ' '.join([word['text'] for word in line_words]).strip()
                    if len(line_text) < 2:
                        continue
                    
                    avg_size = np.mean([word.get('size', 12) for word in line_words])
                    font_names = [word.get('fontname', '') for word in line_words]
                    most_common_font = max(set(font_names), key=font_names.count) if font_names else ''
                    
                    line_info = {
                        'text': line_text,
                        'page': page_num,
                        'y_coord': y_coord,
                        'x_coord': min([word['x0'] for word in line_words]),
                        'font_size': avg_size,
                        'font_name': most_common_font,
                        'bbox': {
                            'x0': min([word['x0'] for word in line_words]),
                            'y0': min([word['top'] for word in line_words]),
                            'x1': max([word['x1'] for word in line_words]),
                            'y1': max([word['bottom'] for word in line_words])
                        }
                    }
                    lines.append(line_info)
        return lines
    
    def compute_features(self, lines: List[Dict]) -> pd.DataFrame:
        """Compute features for each line"""
        features = []
        pages = {}
        for line in lines:
            pages.setdefault(line['page'], []).append(line)
        
        for line in lines:
            page_num = line['page']
            page_lines = pages[page_num]
            
            text = line['text']
            word_count = len(text.split())
            char_count = len(text)
            uppercase_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
            
            page_font_sizes = [l['font_size'] for l in page_lines]
            font_size_rank = sorted(set(page_font_sizes), reverse=True).index(line['font_size']) + 1
            font_size_percentile = np.percentile(page_font_sizes, 75) if len(page_font_sizes) > 1 else line['font_size']
            is_large_font = line['font_size'] >= font_size_percentile
            
            page_x_coords = [l['x_coord'] for l in page_lines]
            median_x = np.median(page_x_coords)
            indentation = abs(line['x_coord'] - median_x)
            
            y_coords = sorted([l['y_coord'] for l in page_lines])
            line_y_idx = y_coords.index(line['y_coord'])
            space_above = abs(y_coords[line_y_idx] - y_coords[line_y_idx - 1]) if line_y_idx > 0 else 0
            space_below = abs(y_coords[line_y_idx + 1] - y_coords[line_y_idx]) if line_y_idx < len(y_coords) - 1 else 0
            
            has_numbering = bool(re.match(r'^\d+[\.\):]', text.strip()))
            has_sub_numbering = bool(re.match(r'^\d+\.\d+', text.strip()))
            is_all_caps = text.isupper() and len(text) > 3
            ends_with_colon = text.strip().endswith(':')
            
            is_bold = 'bold' in line['font_name'].lower() or 'Bold' in line['font_name']
            is_italic = 'italic' in line['font_name'].lower() or 'Italic' in line['font_name']
            
            is_first_page = page_num == 0
            relative_position = (page_num - 1) / max(len(pages), 1)
            
            feature_dict = {
                'word_count': word_count,
                'char_count': char_count,
                'uppercase_ratio': uppercase_ratio,
                'font_size': line['font_size'],
                'font_size_rank': font_size_rank,
                'is_large_font': int(is_large_font),
                'indentation': indentation,
                'space_above': space_above,
                'space_below': space_below,
                'has_numbering': int(has_numbering),
                'has_sub_numbering': int(has_sub_numbering),
                'is_all_caps': int(is_all_caps),
                'ends_with_colon': int(ends_with_colon),
                'is_bold': int(is_bold),
                'is_italic': int(is_italic),
                'is_first_page': int(is_first_page),
                'relative_position': relative_position,
                'page': page_num,
                'text': text
            }
            features.append(feature_dict)
        return pd.DataFrame(features)
    
    def predict_outline(self, pdf_path: str) -> Dict:
        """Extract structured JSON from a PDF"""
        if self.classifier is None:
            raise ValueError("Model not trained! Call load_model first.")
        
        lines = self.extract_text_with_layout(pdf_path)
        if not lines:
            return {"pages": []}
        
        lines_df = self.compute_features(lines)
        
        X = lines_df[self.feature_columns]
        y_pred_encoded = self.classifier.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred_encoded)
        
        pages_dict = {}
        current_section = None
        current_sub_section = None
        
        for idx, (_, row) in enumerate(lines_df.iterrows()):
            label = y_pred[idx]
            page_num = int(row['page']) + 1  # 1-based
            
            if page_num not in pages_dict:
                pages_dict[page_num] = {"page_number": page_num, "content": []}
            
            if label == "Title":
                continue
            elif label == "H1":
                current_section = row['text']
                current_sub_section = None
            elif label in ["H2", "H3"]:
                current_sub_section = row['text']
            else:
                content_item = {
                    "type": "paragraph",
                    "section": current_section,
                    "sub_section": current_sub_section,
                    "text": row['text']
                }
                pages_dict[page_num]["content"].append(content_item)
        
        return {"pages": list(pages_dict.values())}
    
    def load_model(self, model_path: str):
        """Load a trained model"""
        model_data = joblib.load(model_path)
        self.classifier = model_data['classifier']
        self.label_encoder = model_data['label_encoder']
        self.feature_columns = model_data['feature_columns']


# ===============================
# 📌 Usage in Kaggle
# ===============================

# Path to your files in Kaggle
MODEL_PATH = "/kaggle/input/random-forest-heading-classifier/other/default/1/model.pkl"
PDF_PATH = "/kaggle/input/test23231/testpdf.pdf"

# Initialize and load model
extractor = PDFOutlineExtractor()
extractor.load_model(MODEL_PATH)

# Predict structured JSON
result = extractor.predict_outline(PDF_PATH)

# Save to output.json
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

# Show sample result
print(json.dumps(result, indent=2, ensure_ascii=False))


{
  "pages": [
    {
      "page_number": 1,
      "content": [
        {
          "type": "paragraph",
          "section": null,
          "sub_section": null,
          "text": "MONTHLY"
        },
        {
          "type": "paragraph",
          "section": "June 2025",
          "sub_section": null,
          "text": "June 2025 Mutual Fund investments are subject to market risks, read all scheme related documents carefully. Page |"
        }
      ]
    },
    {
      "page_number": 2,
      "content": [
        {
          "type": "paragraph",
          "section": "Macro-Economic Indicators May-25 Apr-25 Mar-25 Feb-25 Jan-25 Dec-24",
          "sub_section": "Consumption",
          "text": "-16.7 11.4 -9.0 2.1 -8.8"
        },
        {
          "type": "paragraph",
          "section": "Macro-Economic Indicators May-25 Apr-25 Mar-25 Feb-25 Jan-25 Dec-24",
          "sub_section": "Two-wheeler sales (%YoY)",
          "text": "-5.9 -4.3 -8.3 -2.1 8.9"
        },
        {
   

In [18]:
from pathlib import Path
import json

extractor = PDFContentExtractor()

# Load pre-trained model
extractor.load_model("/kaggle/input/random-forest-heading-classifier/other/default/1/model.pkl")

# Path to your input PDF
pdf_file = "/kaggle/input/test23231/testpdf.pdf"

# Parse PDF
result = extractor.parse_pdf_to_json(pdf_file)

# Save result
output_dir = Path("./output")
output_dir.mkdir(exist_ok=True)
output_file = output_dir / "my_test.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print(f"✅ Saved result to {output_file}")
print(f"Pages parsed: {len(result['pages'])}")


✅ Model loaded from /kaggle/input/random-forest-heading-classifier/other/default/1/model.pkl
📄 Processing /kaggle/input/test23231/testpdf.pdf...
✅ Saved result to output/my_test.json
Pages parsed: 17
